# EfficientNet Lite in Tensorflow

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers as ly
import math

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [6]:
def bn_act(x, bn=True, act=True):
    if bn:
        x = ly.BatchNormalization(epsilon=1e-3,momentum=0.999)(x)
    if act:
        x = ly.Activation(tf.nn.relu6)(x)
    return x

In [3]:
def MBConv(x, n_in, n_out, expansion, ks=3, strides=1, dropout=0.1, r=24):
    residual = x
    skip_connection = (strides == 1) and (n_in == n_out)
    padding = (ks-1)//2
    if expansion != 1:
        # Expand Pointwise
        x = ly.Conv2D(expansion * n_in, kernel_size=1, padding='same', use_bias=False,
                      activation=None)(x)
        x = bn_act(x)
    ## Depthwise
    if strides == 2:
        x = ly.ZeroPadding2D(padding=padding)(x)
    x = ly.DepthwiseConv2D(kernel_size=ks, strides=strides, activation=None, use_bias=False, 
                          padding='same' if strides == 1 else 'valid')(x)
    x = bn_act(x)
    x = ly.Conv2D(n_out, (1, 1), padding='same', activation=None, use_bias=False)(x)
    x = bn_act(x, act=False)
    if skip_connection:
        x = ly.Dropout(0.2)(x)
        x = ly.add([x, residual])
    return x

In [4]:
### Obtained from Paper ###
widths = [32, 16, 24, 40, 80, 112, 192, 320, 1280]
depths = [1, 2, 2, 3, 3, 4, 1]
kernel_sizes = [3, 3, 5, 3, 5, 5, 3]
strides = [1, 2, 2, 2, 1, 2, 1]

In [5]:
def scale_width(w, w_factor):
    w *= w_factor
    new_w = (int(w+4) // 8) * 8
    new_w = max(8, new_w)
    if new_w < 0.9*w:
        new_w += 8
    return int(new_w)

In [7]:
def efficientnet_scaler(w_factor=1, d_factor=1):
    """
    Efficientnet scaler function as defined in the paper.
    """
    scaled_widths = [scale_width(w, w_factor) for w in widths]
    scaled_widths[0] = 32
    scaled_widths[-1] = 1280
    scaled_depths = [math.ceil(d_factor*d) for d in depths]
    scaled_depths[0] = scaled_depths[-1] = 1
    return scaled_widths, scaled_depths

In [22]:
def EfficientNet(w_factor=1, d_factor=1, n_classes=1000, image_size=224):
    scaled_widths, scaled_depths = efficientnet_scaler(w_factor=w_factor, d_factor=d_factor)
    inputs = keras.Input(shape=(image_size, image_size, 3))
    x = ly.ZeroPadding2D(
      padding=1)(inputs)
    x = ly.Conv2D(scaled_widths[0], (3, 3), strides=(2, 2), padding='same', use_bias=False)(inputs)
    x = bn_act(x)
    
    for i in range(7):
        depth = scaled_depths[i]
        stride = strides[i]
        w_in = scaled_widths[i]
        w_out = scaled_widths[i + 1]
        ks = kernel_sizes[i]
        x = MBConv(x, w_in, w_out, expansion= 1 if i == 0 else 6, ks=ks, strides=stride, r= 4 if i==0 else 24)
        for j in range(1, depth):
            x = MBConv(x, w_out, w_out, expansion= 1 if i == 0 else 6, ks=ks, r= 4 if i==0 else 24)
    
    x = ly.Conv2D(scaled_widths[-1], kernel_size=1, use_bias=False)(x)
    x = bn_act(x)
    x = ly.GlobalAveragePooling2D()(x)
    x = ly.Dense(n_classes)(x)
    return keras.Model(inputs=inputs, outputs=x, name="efficientnet")

In [23]:
def efficientnet_lite0(n_classes=1000, builder = EfficientNet):
    return builder(n_classes=n_classes, image_size=224)

In [24]:
def efficientnet_lite1(n_classes=1000, builder = EfficientNet):
    return builder(1, 1.1, n_classes=n_classes, image_size=240)

In [25]:
def efficientnet_lite2(n_classes=1000, builder = EfficientNet):
    return builder(1.1, 1.2, n_classes=n_classes, image_size=260)

In [26]:
def efficientnet_lite3(n_classes=1000, builder = EfficientNet):
    return builder(1.2, 1.4, n_classes=n_classes, image_size=280)

In [27]:
def efficientnet_lite4(n_classes=1000, builder = EfficientNet):
    return builder(1.4, 1.8, n_classes=n_classes, image_size=300)

In [28]:
lite0 = efficientnet_lite0()
lite1 = efficientnet_lite1()
lite2 = efficientnet_lite2()
lite3 = efficientnet_lite3()
lite4 = efficientnet_lite4()

In [41]:
%%time
inp = np.random.random((1, 224, 224, 3))
lite0(inp).shape

CPU times: user 73.5 ms, sys: 30.4 ms, total: 104 ms
Wall time: 79.8 ms


TensorShape([1, 1000])

In [42]:
%%time
inp = np.random.random((1, 300, 300, 3))
lite4(inp).shape

CPU times: user 65.3 ms, sys: 15.1 ms, total: 80.4 ms
Wall time: 70.1 ms


TensorShape([1, 1000])

In [33]:
models = [lite0, lite1, lite2, lite3, lite4]

In [34]:
def fmat(n):
    return "{:.2f}M".format(n / 1e6)

In [35]:
def params(model, f = True):
    count = int(np.sum([np.prod(p.shape) for p in model.variables]))
    return fmat(count) if f else count

In [36]:
for m in models:
    print(params(m))

4.69M
5.47M
6.15M
8.27M
13.12M


**Untested**